In [1]:
using Iterators
using Pipe
using Compat

macro printval(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr," = ", $ee)))
end

macro pz(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr,"\t\t",typeof($ee), "\t", size($ee))))
end

push!(LOAD_PATH, ".")

3-element Array{ByteString,1}:
 "/home/ubuntu/build/julia-master/usr/local/share/julia/site/v0.5"
 "/home/ubuntu/build/julia-master/usr/share/julia/site/v0.5"      
 "."                                                              

In [3]:
using WordEmbeddings

In [ ]:
words = Set(collect(1:9))

In [ ]:
isempty(Set{String}())

In [ ]:
bigram = zeros(length(words),length(words))
bigram[1,2:end]=1./(length(words)-1)
bigram[2,4]=0.5
bigram[2,8]=0.5
bigram[3,3]=0.5
bigram[3,4:8]=0.1
bigram[4,3]=0.7
bigram[4,4]=0.1
bigram[4,5]=0.1
bigram[4,6]=0.1
bigram[5,4]=0.1
bigram[5,6]=0.2
bigram[5,2]=0.4
bigram[5,8]=0.2
bigram[5,9]=0.1
bigram[6,7]=1.0
bigram[7,5]=0.5
bigram[7,7]=0.5
bigram[8,8]=0.2
bigram[8,9]=0.8
bigram[9,:]=0
bigram=bigram' #I Wrote this in the slow order, it should be Dest,source

In [ ]:
function select_node(x, edge_probs,ϵ=10.0^-5)
    @assert any(edge_probs[:, x] .> ϵ)
    cutoff = rand()
    total = 0.0
    for y in 1:size(edge_probs)[1]
        total+=edge_probs[y,x]
        if total>=cutoff
            return y
        end
    end
    
    @assert(total>=cutoff, "x=$x, total= $total, cutoff=$cutoff") #Should Never Reach this point
end


In [ ]:
typealias Ant{T} Vector{T}

function run_ant(edge_probs,ϵ=10.0^-5)
    path = [1] #1 is always the start node. Attached to it are unigram probs
    while any(edge_probs[:, path[end]] .> ϵ ) #While there is anywhere to go from here
        push!(path, select_node(path[end], edge_probs, ϵ))
    end
    path    
end


function run_ants(τ,η, α, β,n_ants)
    # τ = Pheromone
    # η = Prior
    # α = pheremone weight
    # ϵ = Cut-off for a edge to be considered disconnected
    
    edges = τ.^α .* η.^β
    
    edges./=sum(edges,1) #Normalise to probabilities
    edges[isnan(edges)]=0.0 #The terminal node will have a sum of zero
    map( _ -> run_ant(edges), 1:n_ants)   
end

function optimise(η, α, β, n_ants, n_gens)
    
    assert(α>=1.0)
    assert(β>=1.0)
    
    max_fit = -Inf;
    fittest_path = [NaN]
    τ = ones(η)
    for gen_ii in 1:n_gens
        print("Gen: $gen_ii\n")
        paths = run_ants(τ,η, α, β, n_ants)
        
        function fitness(path)
            # a overly simple fitness function
            1./abs(30 - sum(path))
        end
        
    
        for path in paths
            fit = fitness(path)
            if fit>max_fit
                max_fit = fit
                fittest_path = path
            end
            
            for node_ii in 1:length(path)-1
                node = path[node_ii]
                next_node = path[node_ii+1]
                τ[next_node, node] += fit
            end
        end
    end #Next Generation
    
    fittest_path, max_fit
    
end

In [ ]:
optimise(bigram,1,2, 10,10)

In [ ]:
sum(ans[1])

In [ ]:
#`git add AntSubsetSub.ipynb` |> run
#`git commit -m="Fixed incorrect fitness inversion"` |> run
`git push` |> run